In [443]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd")

In [444]:
#Read the data in
nyc_crime_raw = pd.read_csv('nyc_crime_2014.csv')

### Describe the Data

In [445]:
#Take a quick glance at the data
nyc_crime_raw.head(10)

,City,Population,ViolentCrime,Murder,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,Unnamed: 12
0,Adams Village,"1,851",0,0.000,NaN,0,0,11,1,10,0,0.000,nan
1,Addison Town and Village,"2,568",2,0.000,NaN,1,1,49,1,47,1,0.000,nan
2,Afton Village4,820,0,0.000,0,0,0,1,0,1,0,0.000,nan
3,Akron Village,"2,842",1,0.000,NaN,0,1,17,0,17,0,0.000,nan
4,Albany4,"98,595",802,8.000,54,237,503,"3,888",683,"3,083",122,12.000,nan
5,Albion Village4,"5,872",26,0.000,3,2,21,204,41,159,4,0.000,nan
6,Alexandria Bay Village4,"1,107",0,0.000,0,0,0,7,2,5,0,0.000,nan
7,Alfred Village4,"4,032",11,1.000,1,0,9,30,6,24,0,0.000,nan
8,Altamont Village4,"1,723",1,0.000,0,0,1,2,2,0,0,0.000,nan
9,Amherst Town4,"118,860",128,1.000,16,43,68,"2,066",176,"1,846",44,2.000,nan


In [446]:
#Remove superfluous column
nyc_crime_raw.drop(columns=['Unnamed: 12'], inplace=True)

In [447]:
nyc_crime_raw.shape

(376, 12)

In [448]:
nyc_crime_raw.describe()

,Murder,Arson
count,369.000,365.000
mean,1.453,1.425
std,17.694,7.995
min,0.000,0.000
25%,0.000,0.000
50%,0.000,0.000
75%,0.000,1.000
max,333.000,135.000


### Clean the Data (remove strings, NaN's, and convert to int)

In [449]:
#Now that we know we have 376 observations across 12 columns, let's Find NaN's by column

missing_values_count = nyc_crime_raw.isnull().sum()
print(missing_values_count)

City                   1
Population             7
ViolentCrime           7
Murder                 7
Rape                 149
Robbery                7
AggravatedAssault      7
PropertyCrime          8
Burglary               7
LarcenyTheft           8
MotorVehicleTheft      7
Arson                 11
dtype: int64


**Drop NaN's in the columns we care most about**

In [450]:
nyc_crime_raw = nyc_crime_raw.dropna(subset=['Population', 'Robbery', 'Burglary', 'PropertyCrime'])

**Remove commas from all columns of interest**

In [451]:
nyc_crime_raw['Population'] = nyc_crime_raw['Population'].apply(lambda x: str(x).replace(',', ''))

In [452]:
nyc_crime_raw['Robbery'] = nyc_crime_raw['Robbery'].apply(lambda x: str(x).replace(',', ''))

In [453]:
nyc_crime_raw['Burglary'] = nyc_crime_raw['Burglary'].apply(lambda x: str(x).replace(',', ''))

In [454]:
nyc_crime_raw['PropertyCrime'] = nyc_crime_raw['PropertyCrime'].apply(lambda x: str(x).replace(',', ''))

In [455]:
nyc_crime_raw['LarcenyTheft'] = nyc_crime_raw['LarcenyTheft'].apply(lambda x: str(x).replace(',', ''))

**Change columns to int**

In [456]:
nyc_crime_raw['Population'] = nyc_crime_raw['Population'].astype(int)

In [457]:
nyc_crime_raw['Robbery'] = nyc_crime_raw['Robbery'].astype(int)

In [458]:
nyc_crime_raw['Burglary'] = nyc_crime_raw['Burglary'].astype(int)

In [459]:
nyc_crime_raw['PropertyCrime'] = nyc_crime_raw['PropertyCrime'].astype(int)

In [460]:
nyc_crime_raw['LarcenyTheft'] = nyc_crime_raw['LarcenyTheft'].astype(int)

### Purge outliers and potential skewness

In [461]:
print(nyc_crime_raw['PropertyCrime'].describe())
print(nyc_crime_raw['Population'].describe())
print(nyc_crime_raw['Robbery'].describe())
print(nyc_crime_raw['Burglary'].describe())
print(nyc_crime_raw['LarcenyTheft'].describe())

count      368.000
mean       698.361
std       7123.614
min          0.000
25%         25.000
50%         76.000
75%        271.500
max     135747.000
Name: PropertyCrime, dtype: float64
count       368.000
mean      37888.399
std      441757.416
min          79.000
25%        2628.250
50%        6564.500
75%       15534.750
max     8473938.000
Name: Population, dtype: float64
count     368.000
mean       60.823
std       867.655
min         0.000
25%         0.000
50%         1.000
75%         4.000
max     16581.000
Name: Robbery, dtype: float64
count     368.000
mean      101.160
std       856.253
min         0.000
25%         4.000
50%        12.500
75%        39.000
max     15916.000
Name: Burglary, dtype: float64
count      368.000
mean       562.791
std       5869.850
min          0.000
25%         20.000
50%         60.500
75%        228.500
max     112107.000
Name: LarcenyTheft, dtype: float64


**Start by removing data less than the 25th percentile, then removing extreme outliers by using the quantile() method. Do this for all variables.**

In [462]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['PropertyCrime'] > 25]

In [463]:
nyc_crime_raw['PropertyCrime'].quantile(0.95)

2009.4499999999998

In [464]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['PropertyCrime'] < 2009]

In [465]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Population'] > 2628]

In [466]:
nyc_crime_raw['Population'].quantile(0.95)

44738.79999999999

In [467]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Population'] < 66486]

In [468]:
#Robbery is different. The 25th percentile is 0. Let's instead look at a lower quantile
nyc_crime_raw['Robbery'].quantile(0.85)

12.0

In [469]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Robbery'] < 7]

In [470]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Burglary'] > 4]

In [471]:
nyc_crime_raw['Burglary'].quantile(0.95)

61.19999999999999

In [472]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['Burglary'] < 61]

In [473]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['LarcenyTheft'] > 20]

In [474]:
nyc_crime_raw['LarcenyTheft'].quantile(0.95)

349.25

In [475]:
nyc_crime_raw = nyc_crime_raw[nyc_crime_raw['LarcenyTheft'] < 349]

**Create Population Squared Feature**

In [476]:
nyc_crime_raw['PopulationSquared'] = nyc_crime_raw['Population']**2

### Run the model

In [477]:
# Instantiate and fit our model.
regression = linear_model.LinearRegression()
Y = nyc_crime_raw['PropertyCrime']
X = nyc_crime_raw[['Population', 'PopulationSquared', 'Burglary', 'Robbery']]
regression.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regression.coef_)
print('\nIntercept: \n', regression.intercept_)
print('\nR-squared:')
print(regression.score(X, Y))


Coefficients: 
 [ 3.71191288e-03 -6.14920267e-08  4.31643508e+00  7.53440627e+00]

Intercept: 
 2.539791491470311

R-squared:
0.6490968785254018


### Validate the model

In [478]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(regression, X, Y, cv=10)

array([0.81977676, 0.75301842, 0.13816034, 0.26001248, 0.68295977,
       0.82813815, 0.81833987, 0.34091492, 0.32259765, 0.65306851])

In [479]:
# Use a ~ to represent an '=' from the functional form
linear_formula = 'PropertyCrime ~ Population+PopulationSquared+Burglary+Robbery'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=nyc_crime_raw).fit()

In [480]:
lm.params

Intercept            2.540
Population           0.004
PopulationSquared   -0.000
Burglary             4.316
Robbery              7.534
dtype: float64

In [481]:
lm.pvalues

Intercept           0.842
Population          0.086
PopulationSquared   0.355
Burglary            0.000
Robbery             0.012
dtype: float64

In [482]:
lm.rsquared

0.6490968785254017

In [483]:
lm.conf_int()

,0,1
Intercept,-22.611,27.690
Population,-0.001,0.008
PopulationSquared,-0.000,0.000
Burglary,3.551,5.082
Robbery,1.678,13.390


**Takeaway:** Based on the fluctuation seen in the cross-validation above, along with the fact that out of all features, only 'burglary' had a p-value equal to or less than 0.05, we will create a different version of the model below.

### Create revised model

Removed Population and Population Squared, Added LarcenyTheft

In [493]:
# Instantiate and fit our model.
regression = linear_model.LinearRegression()
Y = nyc_crime_raw['PropertyCrime']
X = nyc_crime_raw[['Burglary', 'Robbery', 'LarcenyTheft']]
regression.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regression.coef_)
print('\nIntercept: \n', regression.intercept_)
print('\nR-squared:')
print(regression.score(X, Y))


Coefficients: 
 [1.01978963 0.35822054 1.01386312]

Intercept: 
 0.7318621733487447

R-squared:
0.9991601607971416


In [497]:
# Use a ~ to represent an '=' from the functional form
linear_formula = 'PropertyCrime ~ Burglary+Robbery+LarcenyTheft'

# Fit the model to our data using the formula.
lm = smf.ols(formula=linear_formula, data=nyc_crime_raw).fit()

In [498]:
lm.pvalues

Intercept      0.070
Burglary       0.000
Robbery        0.015
LarcenyTheft   0.000
dtype: float64

In [499]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(regression, X, Y, cv=10)

array([0.99907655, 0.99951516, 0.99905488, 0.99951741, 0.99910037,
       0.99954456, 0.99848302, 0.99490303, 0.9994934 , 0.99917967])